In [1]:
%%capture
import sys
!{sys.executable} -m pip install diskcache==4.1.0

# Caching

The authors stress that careful caching is important for a performant ML workflow.
In chapter 2.10 they introduce the

1. `@functools.lru_cache(1, typed=True)` decorator to cache the output of callables (functions, methods) in memory.
2. `@raw_cache.memoize(typed=True)` decorator to cache the output of callables on disk.
  - `raw_cache` is the created with the `utils.getCache` function and sets up a `diskcache.FanoutCache` in the `data-unversioned/cache/` folder.

## The `diskcache` module

The [diskcache](https://grantjenks.com/docs/diskcache/tutorial.html) module manages
a disk and file backed cache, and its 
[tutorial page](https://grantjenks.com/docs/diskcache/tutorial.html)
highlights the main functionality of the module.

## The `Cache` object

The core object is an instances of the `Cache` class, which
is linked to a directory, either 

- user-specified as the argument to `Cache()` or
- a temporary directory that is automatically created

and keeps track of the cached items (and their attributes) in a SQLite database.

### Creating a cache

A Cache object is created with a call to `Cache()`. 
It must be explictely closed after use.

In [2]:
from diskcache import Cache
cache = Cache()  # use temporary directory
print(cache.directory)
cache.close()

/tmp/diskcache-ihlj76kw


Alternatively, a context manager can be used to ensure that
the connection is closed after use:

In [3]:
with Cache(cache.directory) as reference:
    reference['key'] = 'value'

Closed Cache objects will automatically re-open when accessed, but that's a relatively slow operation.

### Adding & retrieving objects

The [Cache object](https://grantjenks.com/docs/diskcache/api.html#id3)
behaves similar to a python Dictionary:

In [4]:
'key' in cache

True

In [5]:
cache['key']

'value'

In [6]:
cache['key']

'value'

In [7]:
cache['key'] = 'newvalue'
[x for x in cache]

['key']

The `.set()` and `.get()` methods come with additional keyword parameters:

#### [set](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.set)

- `expire`: seconds until item expires
- `read`: read value as bytes from file?
- `tag`: text to associate with key
- `retry`: retry if database timeout occurs 

The following example addsa BytesIO object to the cache,
which needs to be read as a (binary) file.

In [8]:
from io import BytesIO
cache.set('key', BytesIO(b'value'), expire=5, read=True, tag='data')

True

#### [get](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.get)

- `default`: value to return if key is missing
- `read`: return file handle to value?
- `expire_time`: return expire_time in tuple?
- `tag`: return tag in tuple?
- `retry`: retry if database timeout occurs?

In [9]:
result = cache.get('key', read=True, expire_time=True, tag=True)
reader, timestamp, tag = result
print(reader.read().decode())

value


In [10]:
timestamp

1684168934.3760946

In [11]:
print(tag)  # the tag

data


#### [add](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.add)

The `add()` method only adds a key to the cache if it _doesn't exist, yet_. Existing values are _not_ updated.

In [12]:
cache.add('key', 'other_value')
cache['key']  # not updated

b'value'

### Removing items from the cache

There are numerous ways to remove items from the cache, some of which perform
queries on the SQLite database e.g. to identify items matching a specific _tag_.

#### `del`

An individual item can be deleted from the cache with the `del` command:

In [13]:
del cache['key']
cache.get('key', default='Not found')

'Not found'

#### [pop](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.pop)

`.pop()` removes an item from the cache and return its value:

In [14]:
cache['key'] = 'new_key'
cache.pop('key')

'new_key'

#### [clear](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.clear)

Removes _all_ items from cache

In [15]:
cache['key'] = 'a_key'
cache.clear()
[x for x in cache]

[]

#### [evict](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.evict)

Remove items with matching tag from cache. (The lookup of tags can be sped up by 
creating an index, by setting `Cache(tag_index=True)` when the cache is created or
via the `.create_tag_index()` method on an existing cache.

In [16]:
cache.set('key1', 'yet_another_key', tag = 'first')
cache.set('key2', 'yup,another key', tag = 'second')
[x for x in cache]

['key1', 'key2']

In [17]:
cache.evict(tag = 'first')
[x for x in cache]

['key2']

### Timeout

The `timeout` parameter, sets a limit on how long to wait for database transactions
before raising a `diskcache.Timeout` error is raised. The default timeout is 0.010 (10 milliseconds).

## The `FanoutCache`

This Class automatically shards the underlying database, e.g. it splits the SQLite
database into multiple smaller ones to decrease blocking writes to any single one.

A shard for every concurrent writer is suggested, the default is 8.

As opposed to the `Cache`, a `FanoutCache` catches all timeout errors and aborts the operation. As a result, set and delete methods may silently fail.

For example, the following creates a cache in a temporary directory with four shards and a one second timeout:

In [18]:
from diskcache import FanoutCache
cache = FanoutCache(shards=4, timeout=1)

### Memoization

Caches have an additional feature: the `@memoize` decorator that caches arguments and return values of _callables_ (e.g. functions and methods). 
It is similar to the `functools.lru_cache` decorator (and accepts the same
arguments).

This decorator can be used to add the results of calls to the (on-disk) cache.
(Remember to include parentheses when adding the decorator: `@cache.memoize()`.

In [19]:
@cache.memoize(typed=True, expire=1, tag='fib')
def fibonacci(number):
    if number == 0:
        return 0
    elif number == 1:
        return 1
    else:
        return fibonacci(number - 1) + fibonacci(number - 2)
print(sum(fibonacci(value) for value in range(100)))

573147844013817084100


## Deque and Index classes

A `Deque` is a `collections.deque`-compatible double-ended queue, but - because it
stores the objects on disk - it requires a constant amount of memory.

Use the `push`, `pull`, and `peek` methods to interact with a `diskcache.Deque`.

The `diskcache.Index` provides a mutable mapping and _ordered dictionary_ interface.
Index objects inherit all the benefits of Cache objects but never evict or expire items.

## [Disk](https://grantjenks.com/docs/diskcache/tutorial.html#disk)

All objects are serialized and deserialized as `diskcache.Disk` objects.
Keys are always stored in the cache metadata database while values are sometimes stored separately in files. 

Four data types can be stored natively in the cache metadata database: `integers`, `floats`, `strings`, and `bytes`. Other datatypes are converted to bytes via the [Pickle protocol](https://docs.python.org/3/library/pickle.html).

To use serialization types other than Pickle, you may pass in a Disk subclass to initialize the cache using different file types. For example, the following `JSONDisk` class uses compressed JSON files instead. (The 
[JSONDisk class](https://grantjenks.com/docs/diskcache/api.html#jsondisk) is
included in the `diskcache` module as well.)

In [20]:
from diskcache import Disk, UNKNOWN
class JSONDisk(Disk):
    def __init__(self, directory, compress_level=1, **kwargs):
        self.compress_level = compress_level
        super().__init__(directory, **kwargs)

    def put(self, key):
        json_bytes = json.dumps(key).encode('utf-8')
        data = zlib.compress(json_bytes, self.compress_level)
        return super().put(data)

    def get(self, key, raw):
        data = super().get(key, raw)
        return json.loads(zlib.decompress(data).decode('utf-8'))

    def store(self, value, read, key=UNKNOWN):
        if not read:
            json_bytes = json.dumps(value).encode('utf-8')
            value = zlib.compress(json_bytes, self.compress_level)
        return super().store(value, read, key=key)

    def fetch(self, mode, filename, value, read):
        data = super().fetch(mode, filename, value, read)
        if not read:
            data = json.loads(zlib.decompress(data).decode('utf-8'))
        return data

with Cache(disk=JSONDisk, disk_compress_level=6) as cache:
    pass

## [Settings](https://grantjenks.com/docs/diskcache/tutorial.html#settings)

Additional settings can be specified to improve performance of a cache, including

- `size_limit`: 1 Gb by default
- `cull_limit`: The maximum number of keys to cull when adding a new item.
- `statistics`: Collect cache statistics?
- `tag_index`: Create and index for tags?
- `eviction_policy`: Determines the [eviction policy](https://grantjenks.com/docs/diskcache/tutorial.html#eviction-policies).

## Errors

- When the disk or database is full, a `sqlite3.OperationalError` will be raised.


## Caching in Part 2

In the second part of the book, the authors use a helper module `disk.py` that defines:

1. The custom `GzipDisk` class, a subclass of `diskcache.Disk`, to store items as gzip compressed files
  - Files are split into chunks of 1 Gb (2e30 bytes) because python 2 had problems managing larger files. The chunks are recombined when 
    an item is retrieved from the cache.
2. The `getCache()` function that creates a `FanoutCache` object:
  - In the `data-unversioned/cache/` folder
  - With 64 shards
  - A timeout of 1 second
  - A size limit of 3e11
